In [1]:
import os
import math
import random
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

from skimage.io import imshow
#from sklearn.model_selection import train_test_split

#import tensorflow as tf

import keras

from keras.preprocessing.image import img_to_array, load_img
from keras.layers import Conv2D
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K

np.random.seed(1337)

Using TensorFlow backend.


In [2]:
#making sure GPU is in use 
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7303117707286321335
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11285289370
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15512445604292645880
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [3]:
# Set some parameters
in_height = 64
in_width = 64
out_height = 256
out_width = 256
color_dim = 3
path_in = '/home/Matthew/image-super-resolution/data/imagenet/10k/res64/'
path_out = '/home/Matthew/image-super-resolution/data/imagenet/10k/res256/'

In [4]:
#todo: add random shuffle 
#, seed = 0

def load_images(path_in, path_out, test_size):
    images = next(os.walk(path_in))[2] #oswalk is a generator 
    num_images = len(images)

    splitIdx = int(test_size * num_images)
    testSize = int((test_size) * num_images)
    trainSize = int(math.ceil((1-test_size) * num_images))
    
    X_train = np.zeros((trainSize,in_height,in_width,color_dim), dtype=np.float32)
    X_test = np.zeros((testSize,in_height,in_width,color_dim), dtype=np.float32)
    y_train = np.zeros((trainSize,out_height,out_width,color_dim), dtype=np.float32)
    y_test  = np.zeros((testSize,out_height,out_width,color_dim), dtype=np.float32)

    trainIdx = 0
    testIdx = 0
    for idx, image in enumerate(images):
        image_in_path = os.path.join(path_in,image)
        image_out_path = os.path.join(path_out,image)

        image_in_raw =  load_img(image_in_path, grayscale=False)
        image_out_raw = load_img(image_out_path, grayscale=False)

        #converts image to keras preprocessing image, then divide by 255 to un-invert the images
        image_in = (img_to_array(image_in_raw)).squeeze() / 255 
        image_out = (img_to_array(image_out_raw)).squeeze() / 255          
 
        try:
            if (idx % 1000 == 0):
                print("Stage " + str(idx))
        
            if (idx >= splitIdx):
                #print("trainIdx " + str(trainIdx))
                X_train[trainIdx] = image_in
                y_train[trainIdx] = image_out
                
                trainIdx+=1
            else:
                #print("testIdx " + str(testIdx))
                X_test[testIdx] = image_in
                y_test[testIdx] = image_out
                
                testIdx+=1
                
        except Exception as e: 
            print("\nERROR!!!!")
            print(image_in.shape)
            print(trainIdx)
            
            print(image_out.shape)
            print(testIdx)
            
            print("id "+str(idx))
            print(image_in_path)
            print(e)
           
            print("\n")
            
    return X_train,X_test,y_train,y_test

In [5]:
X_train,X_test,y_train,y_test = load_images(path_in, path_out, .2)

Stage 0
Stage 1000
Stage 2000
Stage 3000
Stage 4000
Stage 5000
Stage 6000
Stage 7000
Stage 8000
Stage 9000


In [6]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.core import Activation
from keras import backend as K
from keras.layers import Conv2D,MaxPooling2D,UpSampling2D
from keras.optimizers import RMSprop

In [7]:
#https://github.com/zeoya/Keras-Super-Resolution/blob/master/myUtils/srcnn.py

In [8]:
#padding output size: see http://cs231n.github.io/convolutional-networks/
# output = (W−F+2P)/S+1   w- input size, f-kernel size, p-padding size 

In [14]:
model = Sequential()
inputShape = (in_height, in_width, color_dim)
model.add(UpSampling2D((4,4),input_shape=inputShape))
model.add(Conv2D(16, (9, 9), padding='same', kernel_initializer="he_normal")) # 256 X 256 X 32
model.add(Activation("relu"))
model.add(Conv2D(8, (7, 7), padding='same', kernel_initializer="he_normal")) # 256 X 256 X 16
model.add(Activation("relu"))
model.add(Conv2D(color_dim, (5, 5), padding='same', kernel_initializer="he_normal")) # 256 X 256 X 3
model.add(Activation("sigmoid"))

In [15]:
'''model = Sequential()
inputShape = (in_height, in_width, color_dim)
model.add(UpSampling2D((4,4),input_shape=inputShape))
model.add(Conv2D(32, (9, 9), padding='same', kernel_initializer="he_normal")) # 256 X 256 X 32
model.add(Activation("relu"))
model.add(Conv2D(16, (7, 7), padding='same', kernel_initializer="he_normal")) # 256 X 256 X 16
model.add(Activation("relu"))
model.add(Conv2D(color_dim, (5, 5), padding='same', kernel_initializer="he_normal")) # 256 X 256 X 3
model.add(Activation("sigmoid"))'''

'model = Sequential()\ninputShape = (in_height, in_width, color_dim)\nmodel.add(UpSampling2D((4,4),input_shape=inputShape))\nmodel.add(Conv2D(32, (9, 9), padding=\'same\', kernel_initializer="he_normal")) # 256 X 256 X 32\nmodel.add(Activation("relu"))\nmodel.add(Conv2D(16, (7, 7), padding=\'same\', kernel_initializer="he_normal")) # 256 X 256 X 16\nmodel.add(Activation("relu"))\nmodel.add(Conv2D(color_dim, (5, 5), padding=\'same\', kernel_initializer="he_normal")) # 256 X 256 X 3\nmodel.add(Activation("sigmoid"))'

In [16]:
model.compile(loss='mean_squared_error', optimizer = RMSprop())

In [17]:
model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
up_sampling2d_2 (UpSampling2 (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 256, 256, 16)      3904      
_________________________________________________________________
activation_4 (Activation)    (None, 256, 256, 16)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 256, 256, 8)       6280      
_________________________________________________________________
activation_5 (Activation)    (None, 256, 256, 8)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 256, 256, 3)       603       
_________________________________________________________________
activation_6 (Activation)    (None, 256, 256, 3)       0         
Total para

In [ ]:
batch_size = 256
epochs = 30

trainHistory = model.fit(X_train, y_train, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test, y_test))

Train on 8000 samples, validate on 2000 samples
Epoch 1/30
8000/8000 [==============================] - 86s 11ms/step - loss: 0.0667 - val_loss: 0.0527
Epoch 2/30
8000/8000 [==============================] - 73s 9ms/step - loss: 0.0410 - val_loss: 0.0259
Epoch 3/30
8000/8000 [==============================] - 73s 9ms/step - loss: 0.0343 - val_loss: 0.0208
Epoch 4/30
8000/8000 [==============================] - 73s 9ms/step - loss: 0.0263 - val_loss: 0.0286
Epoch 5/30
8000/8000 [==============================] - 73s 9ms/step - loss: 0.0239 - val_loss: 0.0240
Epoch 6/30
8000/8000 [==============================] - 73s 9ms/step - loss: 0.0206 - val_loss: 0.0167
Epoch 7/30
8000/8000 [==============================] - 73s 9ms/step - loss: 0.0184 - val_loss: 0.0181
Epoch 8/30
8000/8000 [==============================] - 73s 9ms/step - loss: 0.0199 - val_loss: 0.0149
Epoch 9/30
8000/8000 [==============================] - 73s 9ms/step - loss: 0.0161 - val_loss: 0.0332
Epoch 10/30
8000/8000 [=

In [ ]:
loss = trainHistory.history['loss']
val_loss = trainHistory.history['val_loss']
epochsRange = range(epochs)
plt.figure()
plt.plot(epochsRange, loss, label='Training loss',color='g')
plt.plot(epochsRange, val_loss, label='Validation loss',color='b')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
pred = model.predict(X_test)

In [ ]:
#plt.figure(figsize=(256, 256))
print("Test Images")

for i in range(10):
    #plt.subplot(2, 10, i+1)
    plt.imshow(X_test[i])
    plt.show()
    plt.imshow(pred[i])
    #curr_lbl = y_test[i]
    #plt.title("(Label: " + str(label_dict[curr_lbl]) + ")")
    plt.show()
#plt.figure(figsize=(256, 256))
#print("Reconstruction of Test Images")
#for i in range(10):
    #plt.subplot(2, 10, i+1)
    #plt.imshow(pred[i])  
    #plt.show()

In [ ]:
plt.imshow(pred[0])  